In [275]:
# -*- coding: utf8 -*-
from os.path import expanduser
home = expanduser("~")
nKalaba="K3"
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-%s"%nKalaba
serie=repertoire+"/"
complementPhrases=""
phrasesFichiers=["Phrases","Ordre","Traductions"]
#phrasesFichiers=["Phrases"]
nomComptes=serie+complementPhrases+"Comptes.csv"
nomLexique=serie+"Lexique-S2.pkl"
debug=False

In [276]:
#########################IMPORTS############################################
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml, warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
import pandas as pd

In [277]:
with open(nomLexique, 'rb') as input:
   PFM.lexique = pickle.load(input)

In [278]:
with open(serie+"MorphoSyntax.yaml", 'r') as stream:
    morphosyntax=yaml.load(stream)
contractions=morphosyntax["Contractions"]

In [279]:
def compterLexique(exemplesPhrases):
    debug=0
    lexiquePhrases={}
    lexiqueComptes={}
    for nExemple,exemple in enumerate(exemplesPhrases):
        nPhrase=nExemple+1
        motsExemple=[mot for mot in re.split(ur"[\s',]",exemple) if mot!=""]
        for mot in motsExemple:
            if mot in PFM.lexique.formeLexeme:
                lexeme=PFM.lexique.formeLexeme[mot][0]
                if not lexeme in lexiqueComptes: lexiqueComptes[lexeme]=0
                lexiqueComptes[lexeme]+=1
                if not lexeme in lexiquePhrases: lexiquePhrases[lexeme]=set()
                lexiquePhrases[lexeme].add(nPhrase)
                if debug: print lexeme,
            elif mot in contractions:
                for element in contractions[mot]:
                    lexeme=PFM.lexique.formeLexeme[element][0]
                    if not lexeme in lexiqueComptes: lexiqueComptes[lexeme]=0
                    lexiqueComptes[lexeme]+=1
                    if not lexeme in lexiquePhrases: lexiquePhrases[lexeme]=set()
                    lexiquePhrases[lexeme].add(nPhrase)
                    if debug: print lexeme,
            else:
                lexeme=u"#%s#"%mot
                if not lexeme in lexiqueComptes: lexiqueComptes[lexeme]=0
                lexiqueComptes[lexeme]+=1
                if not lexeme in lexiquePhrases: lexiquePhrases[lexeme]=set()
                lexiquePhrases[lexeme].add(nPhrase)
                if debug: print lexeme,
        if debug: print
    return lexiqueComptes,lexiquePhrases

In [280]:
def stripLex(lex):
    return lex.strip("#")

In [281]:
def printLexPhrases(lexPhrases):
    for lex in sorted(lexPhrases.keys()):
        print stripLex(lex.split(".")[0]),", ".join(["(%d)"%num for num in sorted(lexPhrases[lex])])

In [282]:
def printLexComptes(lexComptes):
    for lex in sorted(lexComptes, key=lexComptes.get,reverse=True):
        print stripLex(lex.split(".")[0]), lexComptes[lex]

In [283]:
dfLexComptes=[]

In [284]:
for nFichier,locFichier in enumerate(phrasesFichiers):
    fichier=serie+complementPhrases+locFichier+".csv"
    with codecs.open(fichier, 'r', encoding="utf8") as input:
       exemplesPhrases = input.readlines()
    exemplesPhrases=[exemple.strip().strip(".") for exemple in exemplesPhrases if not exemple.startswith("#")]
    lexComptes, lexPhrases=compterLexique(exemplesPhrases)
    print fichier
    dfLexComptes.append(pd.DataFrame.from_dict(lexComptes,orient="index"))
    dfLexComptes[nFichier].columns=[locFichier]
    dfLexComptes[nFichier]["lexeme"]=dfLexComptes[nFichier].index
    dfLexComptes[nFichier]=dfLexComptes[nFichier][["lexeme",locFichier]]
    if debug:
        print "lexComptes"
        printLexComptes(lexComptes)
        print
        print "lexPhrases"
        printLexPhrases(lexPhrases)
        print
        print
        print

/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K3/Phrases.csv
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K3/Ordre.csv
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K3/Traductions.csv


In [285]:
#dfLexComptes

In [286]:
df12=dfLexComptes[0].merge(dfLexComptes[1],how="outer").merge(dfLexComptes[2],how="outer")
df12[pd.notnull(df12["Traductions"])].sort_values(by="Ordre")

,lexeme,Phrases,Ordre,Traductions
125,pour,NaN,5.0,3.0
116,quatre.A2,NaN,5.0,7.0
106,entrer.V1.VI,NaN,6.0,5.0
131,chatte.C.N2,NaN,6.0,5.0
100,acheter.V1.VT,NaN,7.0,6.0
98,thé.D.N3,NaN,7.0,9.0
97,viande.A.N3,NaN,7.0,9.0
96,lancer.V1.VT,NaN,7.0,4.0
130,offrir.V1.VT,NaN,7.0,5.0
109,fruit.D.N1,NaN,7.0,8.0


In [287]:
df12.to_csv(path_or_buf=nomComptes,sep="\t",encoding="utf8")

In [288]:
nomComptes

'/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K3/Comptes.csv'